In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
vdf = pd.read_csv('fruitvegprices-2017_2022.csv')

In [3]:
vdf

,category,item,variety,date,price,unit
0,fruit,apples,bramleys_seedling,2022-03-11,2.05,kg
1,fruit,apples,coxs_orange_group,2022-03-11,1.22,kg
2,fruit,apples,egremont_russet,2022-03-11,1.14,kg
3,fruit,apples,braeburn,2022-03-11,1.05,kg
4,fruit,apples,gala,2022-03-11,1.03,kg
...,...,...,...,...,...,...
9642,cut_flowers,alstromeria,indoor,2017-11-03,0.27,stem
9643,cut_flowers,chrysanthemum,indoor_spray,2017-11-03,0.22,stem
9644,cut_flowers,lillies,oriental,2017-11-03,0.70,stem
9645,cut_flowers,narcissus,indoor,2017-11-03,0.06,stem


## Ejercicios

### Ejercicio uno: realice un analisis de valores nulos

En caso de existir valores nulos elimine esos registros

In [4]:
valores_nulos = pd.DataFrame()

for date_ in vdf.loc[ : , "date" ].drop_duplicates():
    vdf_date = vdf.loc[ vdf["date"] == date_ , : ]    
    
    for column_ in vdf_date.columns:        
        percentage =  np.sum(pd.isna( vdf_date.loc[ : , column_ ]  )) / vdf_date.drop_duplicates().shape[0]
        
        dicNull = pd.DataFrame({
            "Date": [ date_ ]
            , "Column" : [ column_ ]
            , "%_Null" : [ percentage ]
        })

        valores_nulos = pd.concat([valores_nulos, dicNull])

valores_nulos  

,Date,Column,%_Null
0,2022-03-11,category,0.0
0,2022-03-11,item,0.0
0,2022-03-11,variety,0.0
0,2022-03-11,date,0.0
0,2022-03-11,price,0.0
...,...,...,...
0,2017-11-03,item,0.0
0,2017-11-03,variety,0.0
0,2017-11-03,date,0.0
0,2017-11-03,price,0.0


### Ejercicio dos: Por categoria, cuantos distintos items hay y cuantos distintos variety hay?

El resultado debe de tener el siguiente formato de presentacion:

In [5]:
pd.DataFrame({
    "category" : ['cut_flowers','fruit']
    , "item_count" : [9,10]
    , "variety_Count" : [10,21]
})

,category,item_count,variety_Count
0,cut_flowers,9,10
1,fruit,10,21


Como nota en un mismo dataframe mostramos todo lo solicitado, usted debe de mostrar todas las categorias

In [6]:
conteo_items_variety = vdf.groupby(["category"],as_index=False).agg({ "item":["count"] , "variety":["count"] }).drop_duplicates()
conteo_items_variety.columns = conteo_items_variety.columns.droplevel(1)
conteo_items_variety.columns = ["category", "item_count", "variety_count"]
conteo_items_variety = conteo_items_variety.sort_values(["category"],ascending=True)
conteo_items_variety

,category,item_count,variety_count
0,cut_flowers,342,342
1,fruit,1992,1992
2,pot_plants,49,49
3,vegetable,7264,7264


### Ejercicio tres: Para la combinacion item, variety muestre el ultimo valor registrado por año.

Ejemplo de como debe de mostrar su respuesta

In [7]:
pd.DataFrame({
    "item" : ['Manzana']
    , "variety" : ['Gala']
    , "year" : [2022]
    , "last_price" :  [3.2]
})

,item,variety,year,last_price
0,Manzana,Gala,2022,3.2


In [8]:
vdf["date"] = pd.to_datetime(vdf["date"])

In [9]:
combinado_item_variety = vdf.loc[ :, ["item","variety"]].drop_duplicates().sort_values(["item","variety"], ascending=True)
combinado_item_variety["year"] = vdf["date"].dt.year
combinado_item_variety["last_price"] = vdf.groupby(["item",],as_index=False)["price"].shift(1)
combinado_item_variety = combinado_item_variety.loc[~pd.isna(combinado_item_variety["last_price"]), :]
combinado_item_variety

,item,variety,year,last_price
3,apples,braeburn,2022,1.14
1,apples,coxs_orange_group,2022,2.05
2,apples,egremont_russet,2022,1.22
4,apples,gala,2022,1.05
461,apples,other_early_season,2021,1.02
5,apples,other_late_season,2022,1.03
320,apples,other_mid_season,2021,0.94
651,beans,dwarf_french_or_kidney,2021,1.43
652,beans,runner_climbing,2021,2.09
16,cabbage,round_green_other,2022,0.41


### Ejercicio cuatro: Segun el ejercicio tres, que tanto aumento o disminuyo el precio de cada item-variety 

Ejemplo de como se debe de mostrar su respuesta

In [10]:
pd.DataFrame({
    "item" : ['Manzana','Manzana','Manzana']
    , "variety" : ['Gala','Gala','Gala']
    , "year" : [2020,2021,2022]
    , "last_price" :  [2.7,2.5,3.2]
    , "delta" : [0, (2.5-2.7), (3.2-2.5)]
})

,item,variety,year,last_price,delta
0,Manzana,Gala,2020,2.7,0.0
1,Manzana,Gala,2021,2.5,-0.2
2,Manzana,Gala,2022,3.2,0.7


Segun nuestro ejemplo dummy para 2021 estuvo 0.2 centavos mas barato y en 2022 estuvo 0.7 centavos caro

Utilice la funcion diff para poder calcular la diferencia.

In [11]:
combinado_item_variety = vdf.loc[ :, ["item","variety"]].sort_values(["item","variety"], ascending=True)
combinado_item_variety["year"] = vdf["date"].dt.year
combinado_item_variety["last_price"] = vdf.groupby(["item",],as_index=False)["price"].shift(1)
combinado_item_variety = combinado_item_variety.loc[~pd.isna(combinado_item_variety["last_price"]), :]
combinado_item_variety["delta"] = combinado_item_variety["last_price"].diff()
combinado_item_variety = combinado_item_variety.loc[~pd.isna(combinado_item_variety["delta"]), :]
combinado_item_variety

,item,variety,year,last_price,delta
7031,alstromeria,indoor,2018,0.33,0.00
7077,alstromeria,indoor,2018,0.27,-0.06
7129,alstromeria,indoor,2018,0.23,-0.04
7182,alstromeria,indoor,2018,0.27,0.04
7231,alstromeria,indoor,2018,0.26,-0.01
...,...,...,...,...,...
9433,watercress,pillow_packs,2017,11.44,0.43
9482,watercress,pillow_packs,2017,11.10,-0.34
9534,watercress,pillow_packs,2017,10.75,-0.35
9586,watercress,pillow_packs,2017,10.75,0.00


### Ejercicio cinco: Cual es precio promedio por item, variety

Ejemplo de como se debe de mostrar su respuesta

In [12]:
pd.DataFrame({
    "item" : ['Manzana']
    , "variety" : ['Gala']
    , "year-month" : ['2021-01']   
    , "avg-price" : [0.5]
})

,item,variety,year-month,avg-price
0,Manzana,Gala,2021-01,0.5


Utilice el método "reindex" para cambiar el orden de las columnas. También utilice el método "strftime()" para convertir los valores del campo date al formato de cadena year-month.

In [13]:
precio_promedio = vdf.groupby(["item","variety"],as_index=False).agg({"price" : ["mean"]}).drop_duplicates()
precio_promedio["year_month"] = vdf["date"].dt.strftime('%Y-%m')
precio_promedio.columns.droplevel(1)
precio_promedio.columns = ['item', 'variety', 'avg_price', 'year_month']
precio_promedio = precio_promedio.reindex(columns=['item', 'variety', 'year_month', 'avg_price'])
precio_promedio

,item,variety,year_month,avg_price
0,alstromeria,indoor,2022-03,0.281207
1,apples,braeburn,2022-03,0.898141
2,apples,bramleys_seedling,2022-03,1.217156
3,apples,coxs_orange_group,2022-03,0.984681
4,apples,egremont_russet,2022-03,1.160000
...,...,...,...,...
80,tomatoes,round,2022-02,1.021479
81,tomatoes,vine,2022-02,1.518955
82,tulips,tulips,2022-02,0.211031
83,turnip,turnip,2022-02,1.232332


### Ejercicio seis: muestre por category,item,variety que tanto % ha aumentado su precio

Ejemplo: si para las manzanas gala el primer precio que se registro fue 2.00 y el ultimo precio que se registro es 4.00 este aumento un 100%

In [14]:
pd.DataFrame({
    "category": ["fruta","fruta"]
    ,"item" : ['Manzana','Manzana']
    , "variety" : ['Gala','egremont_russet']
    , "percentage-change-between-first-and-last-price" : [200,150]       
})

,category,item,variety,percentage-change-between-first-and-last-price
0,fruta,Manzana,Gala,200
1,fruta,Manzana,egremont_russet,150


In [15]:
vdf.columns

Index(['category', 'item', 'variety', 'date', 'price', 'unit'], dtype='object')

In [16]:
porcentaje_aumento_precio = vdf.loc[ :, ["category", "item", "variety", "price"]].sort_values(["category", "item", "variety"],ascending=True).drop_duplicates()
porcentaje_aumento_precio["change price"] = porcentaje_aumento_precio.groupby(["item"],as_index=False)["price"].shift(1)
porcentaje_aumento_precio = porcentaje_aumento_precio.loc[~pd.isna(porcentaje_aumento_precio["change price"]), :]
porcentaje_aumento_precio["% change first and last price"] = ((porcentaje_aumento_precio["change price"] - porcentaje_aumento_precio["price"])/porcentaje_aumento_precio["price"])*100
porcentaje_aumento_precio

,category,item,variety,price,change price,% change first and last price
7031,cut_flowers,alstromeria,indoor,0.27,0.33,22.222222
7077,cut_flowers,alstromeria,indoor,0.23,0.27,17.391304
7182,cut_flowers,alstromeria,indoor,0.26,0.23,-11.538462
7336,cut_flowers,alstromeria,indoor,0.25,0.26,4.000000
7575,cut_flowers,alstromeria,indoor,0.31,0.25,-19.354839
...,...,...,...,...,...,...
9343,vegetable,watercress,pillow_packs,11.01,10.82,-1.725704
9386,vegetable,watercress,pillow_packs,11.44,11.01,-3.758741
9433,vegetable,watercress,pillow_packs,11.10,11.44,3.063063
9482,vegetable,watercress,pillow_packs,10.75,11.10,3.255814


### Ejercicio siete: Segun el ejercicio seis por category cual es el item,variety con mayor aumento de crecimiento 

Segun el ejemplo dummy que se utilizo en el ejercicio seis para la category fruta se mostraria la manzana gala unicamente

In [17]:
aumento_crecimiento  = porcentaje_aumento_precio.groupby(["category"],as_index=False).agg({"item" : ["max",] , "variety" : ["max",] , "% change first and last price" : ["max",]})
aumento_crecimiento.columns.droplevel(1)
aumento_crecimiento.columns = ["category", "item", "variety", "max % change first and last price"]
aumento_crecimiento



,category,item,variety,max % change first and last price
0,cut_flowers,tulips,tulips,400.000000
1,fruit,strawberries,victoria,392.000000
2,pot_plants,poinsettia,9_cm,150.000000
3,vegetable,watercress,yelloworange,497.222222
